In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242
!unzip udea*.zip > /dev/null


import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
print ("(rows, columns))", z.shape)


  0% 0.00/20.1M [00:00<?, ?B/s]
100% 20.1M/20.1M [00:00<00:00, 226MB/s]
(rows, columns)) (692500, 12)


In [29]:
z0 = z[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET','FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE', 'RENDIMIENTO_GLOBAL']]
z0.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONMADRE,FAMI_EDUCACIONPADRE,RENDIMIENTO_GLOBAL
0,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Postgrado,Técnica o tecnológica incompleta,medio-alto
1,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica incompleta,Técnica o tecnológica completa,bajo
2,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,bajo
3,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,Secundaria (Bachillerato) completa,No sabe,alto
4,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,medio-bajo


In [30]:
##FAMI TIENE INTERNET
#Me aseguro que no hayan campos NaN
# Reemplazo de NaN usando .loc[]
z0.loc[:, 'FAMI_TIENEINTERNET'] = z0['FAMI_TIENEINTERNET'].fillna('no info')
# Unifico no info con el campo no, obteniendo representación binaria
internet_map={
    'no info': 0,
    'No': 0,
    'Si': 1
}
# Mapear valores de la columna con .loc[]
z0.loc[:, 'FAMI_TIENEINTERNET'] = np.r_[[internet_map[i] for i in z0['FAMI_TIENEINTERNET']]]
z0['FAMI_TIENEINTERNET'].isna().sum()

0

In [31]:
# Convertimos los estratos a numeros
z0['FAMI_ESTRATOVIVIENDA'] = z0['FAMI_ESTRATOVIVIENDA'].str.extract('(\d+)')
z0['FAMI_ESTRATOVIVIENDA'] = z0['FAMI_ESTRATOVIVIENDA'].fillna(0)


<ipython-input-31-42ea37161385>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z0['FAMI_ESTRATOVIVIENDA'] = z0['FAMI_ESTRATOVIVIENDA'].str.extract('(\d+)')
<ipython-input-31-42ea37161385>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z0['FAMI_ESTRATOVIVIENDA'] = z0['FAMI_ESTRATOVIVIENDA'].fillna(0)


In [27]:
z0['FAMI_ESTRATOVIVIENDA'].unique()

array(['3', '4', '5', '2', '1', 0, '6'], dtype=object)

In [32]:
#Convertimos el valor de la matricula en numeros

cmap = {'Entre 1 millón y menos de 2.5 millones': 1.75,
 'Entre 2.5 millones y menos de 4 millones': 3.25,
 'Menos de 500 mil': .250,
 'Entre 500 mil y menos de 1 millón': .75,
 'Entre 4 millones y menos de 5.5 millones': 4.75,
 'Más de 7 millones': 7.75,
 'Entre 5.5 millones y menos de 7 millones': 6.25,
 'No pagó matrícula': 0,
 'no info': -1}
z0.loc[:,'ESTU_VALORMATRICULAUNIVERSIDAD'] = z0['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('no info')
z0.loc[:,'ESTU_VALORMATRICULAUNIVERSIDAD'] = np.r_[[cmap[i] for i in z0.ESTU_VALORMATRICULAUNIVERSIDAD]]
z0.ESTU_VALORMATRICULAUNIVERSIDAD.value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
1.75,204048
3.25,127430
0.25,80263
0.75,78704
4.75,69736
7.75,68014
6.25,38490
0.0,19528
-1.0,6287


In [17]:
#HORAS DE TRABAJO ESTUDIANTE -> convertimos la columna en unos y ceros (trabaja, no trabaja)

horas_map = {
    'Más de 30 horas':1,
    '0':0,
    'Menos de 10 horas':1,
    'Entre 11 y 20 horas':1,
    'Entre 21 y 30 horas':1
}
#Limpiamos los datos nulos
z0.loc[:, 'ESTU_HORASSEMANATRABAJA'] = z0['ESTU_HORASSEMANATRABAJA'].fillna('0')
z0.loc[:,'ESTU_HORASSEMANATRABAJA'] = np.r_[[horas_map[i] for i in z0['ESTU_HORASSEMANATRABAJA']]]
z0.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONMADRE,FAMI_EDUCACIONPADRE,RENDIMIENTO_GLOBAL
0,6.25,1,3,1,Postgrado,Técnica o tecnológica incompleta,medio-alto
1,3.25,0,3,0,Técnica o tecnológica incompleta,Técnica o tecnológica completa,bajo
2,3.25,1,3,1,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,bajo
3,4.75,0,4,1,Secundaria (Bachillerato) completa,No sabe,alto
4,3.25,1,3,1,Primaria completa,Primaria completa,medio-bajo


In [33]:
#EDUCACION DE LA MADRE A ONE_HOT
z0.FAMI_EDUCACIONMADRE = ['no info' if i in ['No sabe', 'No Aplica'] else i for i in z.FAMI_EDUCACIONMADRE.values]
z0.loc[:, 'FAMI_EDUCACIONMADRE'] = z0['FAMI_EDUCACIONMADRE'].fillna('no info')
x = z0.FAMI_EDUCACIONMADRE.values
FAMI_EDUCACIONMADRE_vals = np.unique(x)
FAMI_EDUCACIONMADRE_onehot_vals = {val: np.eye(len(FAMI_EDUCACIONMADRE_vals))[i] for i,val in enumerate(FAMI_EDUCACIONMADRE_vals)}
FAMI_EDUCACIONMADRE_onehot_vals
FAMI_EDUCACIONMADRE_onehot_enc = np.r_[[FAMI_EDUCACIONMADRE_onehot_vals[i] for i in z0.FAMI_EDUCACIONMADRE]]
FAMI_EDUCACIONMADRE_onehot_enc
FAMI_EDUCACIONMADRE_df = pd.DataFrame(FAMI_EDUCACIONMADRE_onehot_enc, columns=[f"FAMI_EDUCACIONMADRE__{v}" for v in FAMI_EDUCACIONMADRE_onehot_vals])
FAMI_EDUCACIONMADRE_df
z0 = pd.concat([FAMI_EDUCACIONMADRE_df, z0], axis=1).drop('FAMI_EDUCACIONMADRE', axis=1)
z0.shape

<ipython-input-33-5400ebe9974c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z0.FAMI_EDUCACIONMADRE = ['no info' if i in ['No sabe', 'No Aplica'] else i for i in z.FAMI_EDUCACIONMADRE.values]


(692500, 17)

In [35]:
z0.head()

,FAMI_EDUCACIONMADRE__Educación profesional completa,FAMI_EDUCACIONMADRE__Educación profesional incompleta,FAMI_EDUCACIONMADRE__Ninguno,FAMI_EDUCACIONMADRE__Postgrado,FAMI_EDUCACIONMADRE__Primaria completa,FAMI_EDUCACIONMADRE__Primaria incompleta,FAMI_EDUCACIONMADRE__Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE__Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE__Técnica o tecnológica completa,FAMI_EDUCACIONMADRE__Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE__no info,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,RENDIMIENTO_GLOBAL
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.25,Menos de 10 horas,3,1,Técnica o tecnológica incompleta,medio-alto
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.25,0,3,0,Técnica o tecnológica completa,bajo
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.25,Más de 30 horas,3,1,Secundaria (Bachillerato) completa,bajo
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.75,0,4,1,No sabe,alto
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.25,Entre 21 y 30 horas,3,1,Primaria completa,medio-bajo


In [19]:
#Convertir la educacion deL PADRE en one_hot
z0.FAMI_EDUCACIONPADRE = ['no info' if i in ['No sabe', 'No Aplica'] else i for i in z.FAMI_EDUCACIONPADRE.values]
z0['FAMI_EDUCACIONPADRE'] = z0['FAMI_EDUCACIONPADRE'].fillna('no info')
x = z0.FAMI_EDUCACIONPADRE.values
FAMI_EDUCACIONPADRE_vals = np.unique(x)
FAMI_EDUCACIONPADRE_onehot_vals = {val: np.eye(len(FAMI_EDUCACIONPADRE_vals))[i] for i,val in enumerate(FAMI_EDUCACIONPADRE_vals)}
FAMI_EDUCACIONPADRE_onehot_vals
FAMI_EDUCACIONPADRE_onehot_enc = np.r_[[FAMI_EDUCACIONPADRE_onehot_vals[i] for i in z0.FAMI_EDUCACIONPADRE]]
FAMI_EDUCACIONPADRE_onehot_enc
FAMI_EDUCACIONPADRE_df = pd.DataFrame(FAMI_EDUCACIONPADRE_onehot_enc, columns=[f"FAMI_EDUCACIONPADRE__{v}" for v in FAMI_EDUCACIONPADRE_onehot_vals])
FAMI_EDUCACIONPADRE_df
z0 = pd.concat([FAMI_EDUCACIONPADRE_df, z0], axis=1).drop('FAMI_EDUCACIONPADRE', axis=1)
z0.shape

(692500, 27)

In [36]:
#POR ULTIMO CONVERTIMOS LOS ALORES DE RENDIMIENTO_GLOBAL EN NUMEROS
y_col = 'RENDIMIENTO_GLOBAL'

rmap = {'alto': 3, 'bajo':0, 'medio-bajo':1, 'medio-alto':2}
z0[y_col] = [rmap[i] for i in z0[y_col]]


In [21]:
#Separar las colmnas
z0 = z0[sorted(z0.columns)]

X = z0[[c for c in z0.columns if c!=y_col]].values
y = z0[y_col].values
X.shape, y.shape

((692500, 26), (692500,))

In [ ]:
z0.head()

,FAMI_EDUCACIONPADRE__Educación profesional completa,FAMI_EDUCACIONPADRE__Educación profesional incompleta,FAMI_EDUCACIONPADRE__Ninguno,FAMI_EDUCACIONPADRE__Postgrado,FAMI_EDUCACIONPADRE__Primaria completa,FAMI_EDUCACIONPADRE__Primaria incompleta,FAMI_EDUCACIONPADRE__Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE__Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE__Técnica o tecnológica completa,FAMI_EDUCACIONPADRE__Técnica o tecnológica incompleta,...,FAMI_EDUCACIONMADRE__Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE__Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE__Técnica o tecnológica completa,FAMI_EDUCACIONMADRE__Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE__no info,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,RENDIMIENTO_GLOBAL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,6.25,1,3,1,2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,3.25,0,3,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,3.25,1,3,1,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,4.75,0,4,1,3
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.25,1,3,1,1


##Make linear model


In [22]:
from sklearn.model_selection import train_test_split
Xtr, Xts, ytr, yts = train_test_split(X,y, train_size=0.8)
Xtr.shape, Xts.shape, ytr.shape, yts.shape

((554000, 26), (138500, 26), (554000,), (138500,))

In [23]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(Xtr, ytr)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [24]:
preds_tr = lr.predict(Xtr)
preds_ts = lr.predict(Xts)

In [25]:
print (preds_tr[:50])
print (preds_ts[:50])

[0 1 1 3 3 0 3 1 3 3 2 3 3 0 3 3 3 1 0 0 1 2 3 0 3 1 0 0 3 3 2 3 3 1 3 0 1
 2 3 0 0 2 3 2 3 3 0 3 0 2]
[0 0 0 3 2 3 0 0 3 0 3 3 3 0 0 1 0 2 3 2 3 3 2 2 1 0 0 1 3 2 3 0 0 2 0 3 3
 0 0 0 0 0 3 1 3 0 1 2 1 0]


In [26]:
np.mean(preds_tr==ytr), np.mean(preds_ts==yts)

(0.3620902527075812, 0.35870758122743684)